# 앱스토어 리뷰 분석 프로젝트
### [1] iOS 리뷰 수집       ← App Store Scraper
### [2] 구글플레이 리뷰 수집 ← Google Play Scraper
### [3] 전처리 및 병합
### [4] 감성 분석           ← Hugging Face / GPT
### [5] 키워드 추출         ← TF-IDF / YAKE / GPT
### [6] 요약                ← GPT or TextRank
### [7] 시각화 및 종합 리포트

In [8]:
## import library
import trino
import pandas as pd
import numpy as np
import matplotlib.ticker as ticker
import matplotlib.font_manager as fm
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
import seaborn as sns
from matplotlib.colors import Normalize
from matplotlib.cm import ScalarMappable

In [ ]:
## iOS 리뷰 수집 - 실제 피씨에서 보여지는 앱스토어에는 리뷰가 없어서 사용 불가능
from urllib.parse import quote
from app_store_scraper import AppStore

app_name_encoded = quote('멜론-melon')  # URL-safe 인코딩
app = AppStore(country='kr', app_name=app_name_encoded, app_id=415597317)
app.review(how_many=200)

ios_reviews = pd.DataFrame(app.reviews)
ios_reviews['platform'] = 'iOS'

/Library/Frameworks/Python.framework/Versions/3.13/lib/python3.13/site-packages/requests/__init__.py:89: RequestsDependencyWarning: urllib3 (2.4.0) or chardet (3.0.4) doesn't match a supported version!
  warnings.warn("urllib3 ({}) or chardet ({}) doesn't match a supported "
2025-05-08 17:33:04,145 [INFO] Base - Initialised: AppStore('kr', '-eb-a9-9c-eb-a1-a0-melon', 415597317)
2025-05-08 17:33:04,146 [INFO] Base - Ready to fetch reviews from: https://apps.apple.com/kr/app/-eb-a9-9c-eb-a1-a0-melon/id415597317
2025-05-08 17:33:04,473 [ERROR] Base - Something went wrong: Expecting value: line 1 column 1 (char 0)
2025-05-08 17:33:04,474 [INFO] Base - [id:415597317] Fetched 0 reviews (0 fetched in total)


In [ ]:
## 실제 홈페이지에서 크롤링해오기 - 전체리뷰는 확인 불가능하고 주요 리뷰 미리보기만 가능(9개정도)
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.keys import Keys
import time
import pandas as pd

# ChromeDriver 경로 설정
chrome_driver_path = "/Users/samuel.ch/Desktop/Work/chromedriver-mac-arm64/chromedriver"
service = Service(chrome_driver_path)
driver = webdriver.Chrome(service=service)

# 앱스토어 URL
url = "https://apps.apple.com/kr/app/%EB%A9%9C%EB%A1%A0-melon/id415597317?see-all=reviews"
driver.get(url)

# 페이지 로드 대기
time.sleep(5)

# "더보기" 버튼 클릭 및 스크롤
actions = ActionChains(driver)
reviews = []

try:
    while True:
        # 리뷰 데이터 수집
        review_elements = driver.find_elements(By.CLASS_NAME, "we-customer-review")
        for review in review_elements:
            try:
                title = review.find_element(By.CLASS_NAME, "we-customer-review__title").text
                body = review.find_element(By.CLASS_NAME, "we-customer-review__body").text
                rating = review.find_element(By.CLASS_NAME, "we-star-rating").get_attribute("aria-label")
                reviews.append({"title": title, "body": body, "rating": rating})
            except Exception as e:
                print("Error while parsing review:", e)

        # "더보기" 버튼 클릭
        try:
            load_more_button = driver.find_element(By.XPATH, "//button[contains(text(), '더보기')]")
            load_more_button.click()
            time.sleep(3)  # 버튼 클릭 후 대기
        except Exception:
            print("더보기 버튼을 찾을 수 없거나 더 이상 로드할 리뷰가 없습니다.")
            break

except Exception as e:
    print("Error during scraping:", e)

# DataFrame으로 변환
reviews_df = pd.DataFrame(reviews)
print(reviews_df)

# 브라우저 닫기
driver.quit()

더보기 버튼을 찾을 수 없거나 더 이상 로드할 리뷰가 없습니다.
                         title  \
0        개인적으로 생각하는 멜론의 불편했던 점   
1                   노래 처음 들은 날   
2                    뮤직 dna...   
3  상대방 곡 볼 수 있는 기능을 다시 추가해주세용!   
4                    댓글 통합 ㄹㅈㄷ   
5                            렉   
6                         재생목록   
7                         업데이트   
8       아니 근데 진짜 왜 그러세요 …. ㅜ??   
9          플레이리스트 한계치 늘려주세요 ㅠㅠ   

                                                body rating  
0  10개월 정도 쓰고 못써먹겠어서 스포티로 갈아탐 1. 해외 음악 라이센스 제일 불편...    2/5  
1  멜론 5년 넘게 꾸준히 잘 이용하고 있는데요\n지금껏 업데이트로 인해 크게 불편했던...    5/5  
2  뮤직 DNA 진짜 맘에 안 드네요. 정말 불편하고 쓸데 없습니다. 몇 년간 사용해오...    5/5  
3  예전에 여자친구와 서로 실시간으로 듣는 노래와 상대방이 많이 들은 노래를 볼 수 있...    5/5  
4  음악어플의 본질이 뭐임? 음악을 듣고 보는데 이어, 사람들과 소통할 수 있는 곳이라...    5/5  
5  10월 1일 음소거스밍 금지 되자마자 렉이 조오오온나게 걸려요 자꾸 노래 끊기고 멜...    5/5  
6  재생목록이 사라졌습니다. 1000개 가까이 있던 곡들이 오늘 오후에 앱 실행하자마자...    3/5  
7  제발 업데이트 전으로 가게 해주시면 안될까요?\n멜론을 사용하는 이유가 이쁘고 편해...    4/5  
8  저 멜론 7년째 쓰다가 처음 리뷰 다는데요 ㅜㅜ 업

In [16]:
reviews_df['body'][2]

'뮤직 DNA 진짜 맘에 안 드네요. 정말 불편하고 쓸데 없습니다. 몇 년간 사용해오면서 타 음악사이트로 갈아타지 않은 이유는 오직 뮤직 DNA 때문이었고 멜론 사이트를 고집하며 누군가의 학생때 추억이 모두 담겨있는데 한순간에 바뀌니 참 허망하네요. 이 리뷰를 보고 다시 원래의 뮤직 dna 로 바꿔주시리라 굳게 믿는 마음으로 별점 5개 드립니다. 제발 바꿔주세요. 개발자 님 하나도 안 예쁩니다. 불편합니다 전 버전으로 돌려주세요. 간곡히 부탁드립니다.\n더 보기'

In [17]:
pip install google-play-scraper

Note: you may need to restart the kernel to use updated packages.


In [26]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
import time
import pandas as pd

# ChromeDriver 경로 설정
chrome_driver_path = "/Users/samuel.ch/Desktop/Work/chromedriver-mac-arm64/chromedriver"
service = Service(chrome_driver_path)
driver = webdriver.Chrome(service=service)

# 구글 플레이스토어 URL
url = "https://play.google.com/store/apps/details?id=com.iloen.melon&hl=ko"
driver.get(url)

# 페이지 로드 대기
time.sleep(5)

# 스크롤을 내려 리뷰를 로드
for _ in range(10):  # 스크롤을 10번 내림 (필요에 따라 조정)
    driver.execute_script("window.scrollBy(0, 1000);")
    time.sleep(2)

# 리뷰 데이터 수집
reviews = []
review_elements = driver.find_elements(By.XPATH, "//div[@jscontroller='soHxf']")  # 리뷰 컨테이너
for review in review_elements:
    try:
        # 화살표 버튼 클릭 (전체 리뷰 표시)
        try:
            more_button = review.find_element(By.XPATH,"//button[@aria-label='평점 및 리뷰 자세히 알아보기']")  # 화살표 버튼
            more_button.click()
            time.sleep(1)  # 클릭 후 대기
        except Exception:
            pass  # 화살표 버튼이 없는 경우 무시

        # 리뷰 내용과 평점 가져오기
        content = review.find_element(By.XPATH, ".//span[@jsname='bN97Pc']").text  # 리뷰 내용
        rating = review.find_element(By.XPATH, ".//div[@role='img']").get_attribute("aria-label")  # 평점
        reviews.append({"content": content, "rating": rating})
    except Exception as e:
        print("Error while parsing review:", e)

# DataFrame으로 변환
reviews_df = pd.DataFrame(reviews)
print(reviews_df)

# 브라우저 닫기
driver.quit()

Empty DataFrame
Columns: []
Index: []


In [28]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
import time
import pandas as pd

# 크롬 드라이버 설정
chrome_driver_path = "/Users/samuel.ch/Desktop/Work/chromedriver-mac-arm64/chromedriver"
service = Service(chrome_driver_path)
options = Options()
options.add_argument("--lang=ko")
driver = webdriver.Chrome(service=service, options=options)

# 전체 리뷰 페이지 URL
url = "https://play.google.com/store/apps/details?id=com.iloen.melon&showAllReviews=true"
driver.get(url)
time.sleep(5)

# 스크롤 반복 (리뷰 로딩)
for _ in range(20):
    driver.execute_script("window.scrollBy(0, 1000);")
    time.sleep(1.5)

# 리뷰 요소 수집
review_elements = driver.find_elements(By.XPATH, "//div[@jscontroller='soHxf']")

reviews = []
for elem in review_elements:
    try:
        content = elem.find_element(By.XPATH, ".//span[@jsname='V67aGc']").text or \
                  elem.find_element(By.XPATH, ".//span[@jsname='V67aGc']").text
        rating_element = elem.find_element(By.XPATH, ".//div[@role='img']")
        rating_text = rating_element.get_attribute("aria-label")  # 예: "5점 중 4점"
        rating = int(rating_text.split(" ")[-1].replace("점", ""))
        reviews.append({
            "content": content,
            "rating": rating
        })
    except Exception as e:
        print("❌ 리뷰 파싱 오류:", e)

# 결과 확인
df = pd.DataFrame(reviews)
print(df.head())

# 브라우저 종료
driver.quit()

Empty DataFrame
Columns: []
Index: []


In [42]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import time
import pandas as pd

# 크롬 드라이버 설정
chrome_driver_path = "/Users/samuel.ch/Desktop/Work/chromedriver-mac-arm64/chromedriver"
service = Service(chrome_driver_path)
options = Options()
options.add_argument("--lang=ko")
driver = webdriver.Chrome(service=service, options=options)

# 전체 리뷰 페이지 열기
url = "https://play.google.com/store/apps/details?id=com.iloen.melon&hl=ko&showAllReviews=true"
driver.get(url)
time.sleep(5)

# 스크롤 다운 (리뷰 로드 유도)
for _ in range(20):
    driver.execute_script("window.scrollBy(0, 1000);")
    time.sleep(1.5)

# '리뷰 모두 보기' 버튼 대기 후 클릭
try:
    show_all_button = WebDriverWait(driver, 10).until(
        EC.element_to_be_clickable((By.XPATH, "//span[text()='리뷰 모두 보기']/ancestor::button"))
    )
    show_all_button.click()
    time.sleep(3)
except Exception as e:
    print("❌ 리뷰 모두 보기 버튼 클릭 실패:", e)

# 리뷰 컨테이너 선택
review_elements = driver.find_elements(By.XPATH, "//div[@jscontroller='X6C1Be']")

reviews = []
for elem in review_elements:
    try:
        # 리뷰 내용
        content = elem.find_element(By.XPATH, ".//div[@class='RHo1pe']").text
        # 리뷰 평점
        rating_element = elem.find_element(By.XPATH, ".//div[@role='img']")
        rating_text = rating_element.get_attribute("aria-label")  # 예: "5점 중 4점"
        rating = int(rating_text.split(" ")[-1].replace("점", ""))
        reviews.append({
            "content": content,
            "rating": rating
        })
    except Exception as e:
        print("❌ 리뷰 파싱 오류:", e)

# 결과 저장 및 출력
df = pd.DataFrame(reviews)
print(df.head())

# 브라우저 종료
driver.quit()

❌ 리뷰 파싱 오류: invalid literal for int() with base 10: '받았습니다.'
Empty DataFrame
Columns: []
Index: []


In [50]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
import time
import pandas as pd

# ChromeDriver 경로 설정
chrome_driver_path = "/Users/samuel.ch/Desktop/Work/chromedriver-mac-arm64/chromedriver"
service = Service(chrome_driver_path)
driver = webdriver.Chrome(service=service)

# 구글 플레이스토어 URL
url = "https://play.google.com/store/apps/details?id=com.iloen.melon&hl=ko"
driver.get(url)

# 페이지 로드 대기
time.sleep(5)

# 스크롤을 내려 리뷰를 로드
for _ in range(10):  # 스크롤을 10번 내림 (필요에 따라 조정)
    driver.execute_script("window.scrollBy(0, 1000);")
    time.sleep(2)

# '리뷰 모두 보기' 버튼 대기 후 클릭
try:
    show_all_button = WebDriverWait(driver, 10).until(
        EC.element_to_be_clickable((By.XPATH, "//span[text()='리뷰 모두 보기']/ancestor::button"))
    )
    show_all_button.click()
    time.sleep(3)
except Exception as e:
    print("❌ 리뷰 모두 보기 버튼 클릭 실패:", e)

# 리뷰 데이터 수집
reviews = []
review_elements = driver.find_elements(By.XPATH, "//div[@jscontroller='RHo1pe']")  # 리뷰 컨테이너
for review in review_elements:
    try:        
        # 리뷰 내용과 평점 가져오기
        content = review.find_element(By.XPATH, ".//div[@class=h3YV2d]").text  # 리뷰 내용        
        rating_text = review.find_element(By.XPATH, ".//div[@role='img']").get_attribute("aria-label")  # 평점

        # 평점에서 숫자 추출 (예: "별표 5개 만점에 3개를 받았습니다.")
        try:
            rating = int(rating_text.split(" ")[3].replace("개", ""))  # "3개"에서 숫자만 추출
        except (IndexError, ValueError):
            rating = None  # 평점이 없거나 숫자가 아닌 경우 None 처리

        reviews.append({"content": content, "rating": rating})
    except Exception as e:
        print("❌ 리뷰 파싱 오류:", e)

# DataFrame으로 변환
reviews_df = pd.DataFrame(reviews)
print(reviews_df)

# 브라우저 닫기
driver.quit()

Empty DataFrame
Columns: []
Index: []


In [51]:
review_elements

[]

[]